In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv1D, Flatten, Input, Dropout, MaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf

dataset = pd.read_csv("dataset.csv")
print(dataset.head(4))

input_data = dataset.iloc[:, :-1]
output_data = dataset.iloc[:, -1]

ss = StandardScaler()
input_data = pd.DataFrame(ss.fit_transform(input_data), columns=input_data.columns)
print(input_data.shape)

input_data = np.expand_dims(input_data, axis=2)

x_train, x_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=40)

cnn = Sequential()
cnn.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
cnn.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(units=6, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(units=4, activation='relu'))
cnn.add(Dense(units=1, activation='sigmoid'))

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

cnn.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.2, callbacks=[early_stopping])

loss, accuracy = cnn.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')

loss, accuracy = cnn.evaluate(x_train, y_train)
print(f'Train accuracy: {accuracy * 100:.2f}%')

data_point = np.array([[0,59,0,1,0,1,1,0,10]])
data_point_scaled = ss.transform(data_point)
data_point_scaled = np.expand_dims(data_point_scaled, axis=2)

prediction = cnn.predict(data_point_scaled)
binary_prediction = (prediction >= 0.5).astype(int)
print(f'Prediction: {binary_prediction[0][0]}')


   Gender  Age  NS1  IgG  IgM  Area  AreaType  HouseType  District  Outcome
0       0   45    0    0    0     1         0          1         1        0
1       1   17    0    0    1     0         1          1         1        0
2       0   29    0    0    0     0         0          2         1        0
3       0   63    1    1    0     0         1          2         1        1
(1000, 9)
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4472 - loss: 0.9638 - val_accuracy: 0.3562 - val_loss: 0.7235
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4683 - loss: 0.9262 - val_accuracy: 0.4500 - val_loss: 0.7053
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4947 - loss: 0.8511 - val_accuracy: 0.5875 - val_loss: 0.6928
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5514 - loss: 0.8355 - val_accuracy: 0.6812 - val_loss: 0.6832
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5946 - loss: 0.7039 - val_accuracy: 0.7812 - val_lo

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction: 1
